In [181]:
import numpy as np
from hypothesis import given, settings
from hypothesis.strategies import integers, random_module, just, booleans, floats

#from gl0learn.opt import MIO_mosek, mosek_level_values
from gl0learn.synthetic import preprocess, generate_synthetic
from gl0learn import fit
from gl0learn import metrics

In [139]:
import sys
sys.path.append("../tests/utils/")

In [140]:
from utils import random_penalty, random_penalty_values, overlap_covariance_matrix, sample_from_cov

In [431]:
from sys import stdout
from time import perf_counter
from collections import namedtuple
from typing import Optional, Dict

import numpy as np


def diag_index(i):
    return (i + 1) * (i + 2) // 2 - 1


MIOResults = namedtuple("MIOResults",
                        ['theta_hat',
                         'z',
                         't',
                         's',
                         'lg',
                         'residuals',
                         'objective',
                         'elapsed',
                         'upper_bound',
                         'lower_bound',
                         'gap',
                         'status'])


def mosek_level_values(theta: np.ndarray, Y: np.ndarray, int_tol: float = 1e-4):
    """

    Parameters
    ----------
    theta : (p, p) symmetric matrix
        The lower triangular will be selected automatically
    Y: (n, p) samples of from theta inverse
    int_tol : float, default = 1e-4
        tolerance such that any value of theta with absolute value less than int_tol is deemed to be 0.

    Returns
    -------
    theta: (p, p) array
        returns `theta` as passed
    theta_tril: (p*(p+1)//2, ) array
        Lower triangular section of theta including the main diagonal
    z_values: (p*(p+1)//2, ) array
        Indicator matrix of lower triangular section of theta including the main diagonal where:
            Any non zero item of the matrix is located
                AND
            Is not located on the main diagonal.
    s_values: (p*(p+1)//2, ) array
        Derived matrix of triangular section of theta including the main diagonal where the value is:
            theta[i, j]**2 if i != j else 0!
    t_values: (p, ) array
        t_values[i] <- 1/theta[i,i]||Ytheta[:, i]||^2

    lg: (p, ) array
        natural log of the main diagonal
    residuals:
    """
    n, p = Y.shape

    assert theta.shape == (p, p), "Initial Theta must be passed as a (p by p matrix)!"
    np.testing.assert_array_almost_equal(theta, theta.T)

    tril_indicies = np.tril_indices(p, k=0)  # Used to select the lower triangular values including the main diagonal

    # Since mosek keeps main diagonal in the l0 and l2 variables.
    # We create a copy and set diagonal to zero to make l0, and l2 calculations easier!
    theta_no_diag = np.copy(theta)
    np.fill_diagonal(theta_no_diag, 0)

    non_zero_values = np.abs(theta_no_diag) > int_tol

    z_values = non_zero_values[tril_indicies]
    s_values = theta_no_diag[tril_indicies]**2

    t_values = np.linalg.norm(Y@theta, axis=0)**2/np.diag(theta)

    lg_values = np.log(np.diag(theta))
    
    if n > p:
        YtY = np.linalg.cholesky(Y.T @ Y).T
    else:
        YtY = Y
    residuals = YtY@theta

    return theta[tril_indicies], z_values, s_values, t_values, lg_values, residuals


def MIO_mosek(Y, l0, l2, M, mio_gap=1e-4, int_tol=1e-4, maxtime=None, initial_values: Optional[Dict[str, np.ndarray]] = None ):
    start_time = perf_counter()
    n, p = Y.shape
    num_coeffs = p * (p + 1) // 2
    try:
        import mosek.fusion as msk
        import mosek
    except ModuleNotFoundError:
        raise Exception(
            f"`mosek` is not installed. Refer ot installation documentation about how to install `mosek`"
        )

    model = msk.Model()
    model.acceptedSolutionStatus(msk.AccSolutionStatus.Feasible)

    theta_tril = model.variable("theta_tril", num_coeffs, msk.Domain.unbounded())
    s = model.variable("s", num_coeffs, msk.Domain.greaterThan(0))
    z = model.variable("z", num_coeffs, msk.Domain.integral(msk.Domain.inRange(0, 1)))
    t = model.variable("t", p, msk.Domain.greaterThan(0))
    lg = model.variable("lg", p, msk.Domain.unbounded())
    residuals = model.variable("residuals", [min(n, p), p], msk.Domain.unbounded())

    theta = theta_tril.fromTril(p)
    if n <= p:
        expr = msk.Expr.mul(msk.Matrix.dense(Y), theta)
    else:
        C = np.linalg.cholesky(Y.T @ Y)
        expr = msk.Expr.mul(msk.Matrix.dense(C.T), theta)
    model.constraint(msk.Expr.sub(residuals, expr), msk.Domain.equalsTo(0))

    for i in range(p):
        model.constraint(
            msk.Expr.vstack(
                theta_tril.index(diag_index(i)),
                msk.Expr.mul(0.5, t.index(i)),
                residuals.slice([0, i], [min(n, p), i + 1]).reshape(min(n, p)),
            ),
            msk.Domain.inRotatedQCone(),
        )
        model.constraint(
            msk.Expr.vstack(
                theta_tril.index(diag_index(i)), msk.Expr.constTerm(1), lg.index(i)
            ),
            msk.Domain.inPExpCone(),
        )

    z_expr = msk.Expr.constTerm(0)
    s_expr = msk.Expr.constTerm(0)
    for i in range(1, p):
        theta_tmp = theta_tril.slice(diag_index(i - 1) + 1, diag_index(i))
        z_tmp = z.slice(diag_index(i - 1) + 1, diag_index(i))
        s_tmp = s.slice(diag_index(i - 1) + 1, diag_index(i))
        expr = msk.Expr.mul(z_tmp, M)
        model.constraint(msk.Expr.sub(expr, theta_tmp), msk.Domain.greaterThan(0))
        model.constraint(msk.Expr.add(theta_tmp, expr), msk.Domain.greaterThan(0))
        expr = msk.Expr.hstack(msk.Expr.mul(0.5, s_tmp), z_tmp, theta_tmp)
        model.constraint(expr, msk.Domain.inRotatedQCone())
        z_expr = msk.Expr.add(z_expr, msk.Expr.sum(z_tmp))
        s_expr = msk.Expr.add(s_expr, msk.Expr.sum(s_tmp))

    z_expr = msk.Expr.mul(l0, z_expr)
    s_expr = msk.Expr.mul(l2, s_expr)
    t_expr = msk.Expr.sum(msk.Expr.sub(t, lg))

    model.objective(msk.ObjectiveSense.Minimize, msk.Expr.add([t_expr, z_expr, s_expr]))

    model.setSolverParam("log", 0)
    model.setSolverParam("mioTolAbsRelaxInt", int_tol)
    model.setSolverParam("mioTolAbsGap", mio_gap)
    model.setSolverParam("mioTolRelGap", mio_gap)
    model.setSolverParam("mioRelGapConst", 1)

    if maxtime is not None:
        model.setSolverParam("mioMaxTime", maxtime)
    
    if initial_values:
        theta_tril.setLevel(initial_values['theta_tril'])
        s.setLevel(initial_values['s'])
        z.setLevel(initial_values['z'])
        t.setLevel(initial_values['t'])
        lg.setLevel(initial_values['lg'])
        residuals.setLevel(initial_values['residuals'])

    model.solve()
    
    status = model.getProblemStatus()

    lower_bound = model.getSolverDoubleInfo("mioObjBound")
    upper_bound = model.getSolverDoubleInfo("mioObjInt")
    gap = (upper_bound - lower_bound) / max(1, abs(upper_bound))

    return MIOResults(
        theta_hat=theta.level().reshape(p, p),
        z=z.fromTril(p).level().reshape(p, p),
        t=t.level(),
        s=s.level(),
        lg=lg.level(),
        residuals=residuals.level(),
        objective=model.primalObjValue(),
        elapsed=perf_counter() - start_time,
        upper_bound=upper_bound,
        lower_bound=lower_bound,
        gap=gap,
        status=status
    )


In [443]:
p = 100
seed = 2
lXs = {'l0': 0.5, 'l2': 1}
n = 10

In [444]:
model = "independent"
rng = 1
rho = 0.3
normalize="covariance"
x, sigma_truth, theta_truth = generate_synthetic(n, p, model,normalize, rng=rng, rho=rho)
# theta_truth = overlap_covariance_matrix(p=p, seed=seed, decay=.99, max_overlaps=2)
# x = sample_from_cov(cov=theta_truth, n=n, seed=seed)

In [445]:
np.all(np.linalg.eigvalsh(theta_truth) >= 0)

True

In [446]:
theta_truth[0:5, 0:5]

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [447]:

_, _, _, _, Y, _ = preprocess(x, assume_centered=False, cholesky=False)

M = np.max(np.abs(theta_truth*(1-np.eye(p))))
int_tol = 1e-4
#results = MIO_mosek(Y, M=M, l0=lXs['l0'], l2=lXs['l2'], int_tol=int_tol, maxtime=100)

#theta_tril, z, s, t, lg, residuals = mosek_level_values(theta=results.theta_hat, Y=Y, int_tol=int_tol)

In [448]:
results = fit(Y,
    theta_init=None,
    initial_active_set=0.1,
    super_active_set=0.,
    algorithm='CDPSI',
    **lXs,
    check=False,
    scale_x=False,
    max_active_set_size=int(.5*p*(p-1)))

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fitpsi called 
fit 1
fit loop0
current_iter: 1 cur_objective = 65.5358
fit loop1
current_iter: 2 cur_objective = 62.1993
fit loop2
current_iter: 3 cur_objective = 61.402
fit loop3
current_iter: 4 cur_objective = 60.9337
fit loop4
current_iter: 5 cur_objective = 60.7471
fit loop5
current_iter: 6 cur_objective = 60.6518
fit loop6
current_iter: 7 cur_objective = 60.6392
fit loop7
current_iter: 8 cur_objective = 60.6379
fit loop8
current_iter: 9 cur_objective = 60.6376
fit loop9
current_iter: 10 cur_objective = 60.6375
fit loop10
current_iter: 11 cur_objective = 60.6375
Pre psi cost: 60.6375 
PSI iter: 0 
PSI iter: 0 Swapping row: 0
psi_row_fit row =  0 
selected super_active_set start =  {0, 1} 
selected super_active_set end =  {1, 2} 
zero_indices =           1
         2
         3
         4
         5
         6
         7
         8
         9
        10
        11
        12
        13
        14
        15
        16
        17
        18
 

In [450]:
results.active_set_size

[3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 49,
 49,
 49,
 49,
 49,
 49]

In [451]:
print(metrics.false_positives(results.theta, theta_truth))
print(metrics.prediction_error(Y@results.theta, Y@theta_truth))

96
0.7499509001392598


In [452]:
metrics.nonzeros(theta_truth).sum()

100

In [453]:
metrics.nonzeros(results.theta).sum()

196

In [442]:
results.active_set_size

[3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 3771,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 171,
 171,
 171,
 171,
 171,
 171,
 171,
 171,
 171,
 171]

In [389]:
results.costs

[27.643842632089875,
 -1.8847693435883073,
 -16.709647403675827,
 -26.04506149521528,
 -31.654958855799165,
 -34.82943978021072,
 -36.823197844635814,
 -38.058596202845585,
 -38.96829940430466,
 -39.54975386438645,
 -40.0024143881712,
 -40.23288772768855,
 -40.34543911426142,
 -40.41610203510801,
 -40.44685309050708,
 -40.46280004889031,
 -40.47124380359453,
 -40.47614403385928,
 -40.47883911578475,
 -40.48036497453131,
 -40.481239617056566,
 -40.481738976370195,
 -40.482019676666134,
 -40.48217771811441,
 -40.48226756106307,
 -40.4823185127067,
 -40.482347672267764,
 -40.57854639461739,
 -40.60803545218453,
 -40.62690495545458,
 -40.62917046664912,
 -40.62975782741689,
 -40.6299637110526,
 -40.630044072338286,
 -40.63007624285606,
 -40.67756848561106,
 -40.70085040010257,
 -40.70633577295444,
 -40.70844295126828,
 -40.709261996262995,
 -40.70959919994058,
 -40.709736086586744,
 -40.709796648464156,
 -40.70982436313491]

In [454]:
theta_tril, z, s, t, lg, residuals = mosek_level_values(theta=results.theta, Y=Y, int_tol=int_tol)

In [455]:
seeded_results = MIO_mosek(Y, M=M, l0=lXs['l0'], l2=lXs['l2'], int_tol=int_tol, maxtime=10, initial_values={"theta_tril": theta_tril,
                                                                                                            'z': z.astype(float),
                                                                                                            's': s,
                                                                                                            't': t,
                                                                                                            'lg': lg,
                                                                                                            'residuals': residuals.flatten()})

non_seeded_results = MIO_mosek(Y, M=M, l0=lXs['l0'], l2=lXs['l2'], int_tol=int_tol, maxtime=10)

In [458]:
seeded_results.elapsed

0.7332674350036541

In [459]:
non_seeded_results.elapsed

0.669746003986802

In [68]:
import pyomo.environ as pyo
import mosek


In [69]:
from xml.etree import ElementTree as ET

tree = ET.parse('test.opf')

ParseError: syntax error: line 1, column 0 (<string>)

In [72]:
import ast

In [78]:
ast.parse("Hi")